In [2]:
import pynq
import pynq.lib.dma
from pynq import Overlay
from pynq import DefaultIP
from pynq import MMIO
from pynq import Xlnk
import numpy as np
import time
from operator import*
import matplotlib.pyplot as plt

# Load the SPI overlay
ol = Overlay("ad1_filter.bit")

In [ ]:
print(ol.ip_dict)

In [3]:
import matplotlib.pyplot as plt
def plot_to_notebook (time_sec, in_signal, n_samples, out_signal=None):
    plt.figure()
    plt.subplot(1,1,1)
    plt.xlabel('Time (usec)')
    plt.grid()
    plt.plot(time_sec[:n_samples]*1e6, in_signal[:n_samples], 'y-', label='input signal')
    if out_signal is not None:
        plt.plot(time_sec[:n_samples]*1e6, out_signal[:n_samples], 'g-', linewidth=2, label='FIR Output')
    
    plt.legend()

In [4]:
ol?

In [33]:
ad1_ip = ol.PmodAD1_0
dma    = ol.fir_dma

In [34]:
# Function to read data from PmodAD1
def read_adc(channel):
    # Reading raw data from the data register
    register_address = 0x00 + (channel * 0x01)  # Ensure channel data is aligned to 4-byte boundary
    raw_data = ad1_ip.read(register_address)
    
    # Extract the 12-bit ADC value (assuming 12-bit data in 32-bit register)
    adc_value = raw_data & 0x0FFF  # Mask to get the lower 12 bits
    
    return adc_value

In [35]:
adc_data = read_adc(0)
print(f"ADC Data: {adc_data}")

ADC Data: 3623


In [ ]:
T = 0.002
fs = 100e6
n = int(T*fs)
t = np.linspace(0, T, n, endpoint=False)
samples = 10000*np.sin(0.2e6*2*np. pi*t)+ 1500*np.cos (46e6*2*np.pi*t)
samples = samples.astype(np.int32)
print('Number of samples:', len(samples))
plot_to_notebook (t, samples, 1000)

In [46]:
xlnk =Xlnk()
in_buffer = xlnk.cma_array(shape=(n,),dtype=np.int32)
out_buffer = xlnk.cma_array(shape=(n,), dtype =np.int32)

In [ ]:
dma.recvchannel.transfer(out_buffer)
dma.sendchannel.wait()
dma.recvchannel.wait()
stop_time = time.time()


plot_to_notebook(t, samples, 1000, out_signal=out_buffer)
in_buffer.close()
out_buffer.close()
